# Inerpretation-Net Training

## Specitication of Experiment Settings

In [1]:
import sys
print(sys.version)


3.9.6 (default, Aug 18 2021, 19:38:01) 
[GCC 7.5.0]


In [2]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes



config = {
    'function_family': {
        'maximum_depth': 4,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,                      
    },
    'data': {
        'number_of_variables': 15, 
        'num_classes': 2,
        
        'function_generation_type': 'random_vanilla_decision_tree_trained', # 'make_classification' 'random_decision_tree_trained' 'random_decision_tree'
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 1000, #number of samples per function
        #'number_of_generated_datasets': 10000,
        
        'noise_injected_level': 0, 
        'noise_injected_type': 'flip_percentage', # '' 'normal' 'uniform' 'normal_range' 'uniform_range'
    }, 
    'lambda_net': {
        'epochs_lambda': 1000,
        'early_stopping_lambda': True, 
        'early_stopping_min_delta_lambda': 1e-2,
        'batch_lambda': 64,
        'dropout_lambda': 0,
        'lambda_network_layers': [256],
        'optimizer_lambda': 'adam',
        'loss_lambda': 'binary_crossentropy', #categorical_crossentropy
        
        'number_of_lambda_weights': None,
        
        'number_initializations_lambda': 1, 
        
        'number_of_trained_lambda_nets': 10000,
    },     
    
    'i_net': {
        'dense_layers': [2048],
        'convolution_layers': None,
        'lstm_layers': None,
        'dropout': [0],
        
        'optimizer': 'adam', #adam
        'learning_rate': 0.01,
        'loss': 'binary_crossentropy',
        'metrics': ['binary_accuracy'],
        
        'epochs': 200, 
        'early_stopping': True,
        'batch_size': 256,

        'interpretation_dataset_size': 10000,
                
        'test_size': 50, #Float for fraction, Int for number 0
        
        'function_representation_type': 3, # 1=standard representation; 2=sparse representation, 3=vanilla_dt

        'optimize_decision_function': True, #False
        'function_value_loss': True, #False
                      
        'data_reshape_version': None, #default to 2 options:(None, 0,1 2)
        
        'nas': False,
        'nas_type': 'SEQUENTIAL', #options:(None, 'SEQUENTIAL', 'CNN', 'LSTM', 'CNN-LSTM', 'CNN-LSTM-parallel')      
        'nas_trials': 100,
    },    
    
    'evaluation': {   
        #'inet_holdout_seed_evaluation': False,
            
        'random_evaluation_dataset_size': 500, 
        'per_network_optimization_dataset_size': 5000,

        'sklearn_dt_benchmark': False,
        'sdt_benchmark': False,
        
    },    
    
    'computation':{
        'load_model': False,
        
        'n_jobs': -3,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}


## Imports

In [3]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

In [4]:
#######################################################################################################################################
##################################################### IMPORT LIBRARIES ################################################################
#######################################################################################################################################
from itertools import product       
from tqdm import tqdm_notebook as tqdm
import pickle
import numpy as np
import pandas as pd
import scipy as sp
import timeit
import psutil

from functools import reduce
from more_itertools import random_product 
from sklearn.preprocessing import Normalizer

import sys
import os
import shutil

import logging

#from prettytable import PrettyTable
#import colored
import math

import time
from datetime import datetime
from collections.abc import Iterable


from joblib import Parallel, delayed

from scipy.integrate import quad

from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold, KFold, ParameterGrid, ParameterSampler
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, f1_score, mean_absolute_error, r2_score, log_loss
from sklearn.tree import DecisionTreeClassifier, plot_tree



#from similaritymeasures import frechet_dist, area_between_two_curves, dtw
import tensorflow as tf
#import tensorflow_addons as tfa
import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


import tensorflow.keras.backend as K
from livelossplot import PlotLossesKerasTF
#from keras_tqdm import TQDMNotebookCallback

from matplotlib import pyplot as plt
import seaborn as sns


import random 


import warnings

from IPython.display import Image
from IPython.display import display, Math, Latex, clear_output



2021-09-07 08:31:29.863726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-07 08:31:29.863769: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
tf.__version__

'2.6.0'

In [6]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

In [7]:
#######################################################################################################################################
################################################### VARIABLE ADJUSTMENTS ##############################################################
#######################################################################################################################################

config['i_net']['data_reshape_version'] = 2 if data_reshape_version == None and (convolution_layers != None or lstm_layers != None or (nas and nas_type != 'SEQUENTIAL')) else data_reshape_version

#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers if use_gpu else ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
#os.environ['XLA_FLAGS'] =  '--xla_gpu_cuda_data_dir=/usr/lib/cuda-10.1'

logging.getLogger('tensorflow').disabled = True

sns.set_style("darkgrid")
#np.set_printoptions(suppress=True)

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if int(tf.__version__[0]) >= 2:
    tf.random.set_seed(RANDOM_SEED)
else:
    tf.set_random_seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 200)
np.set_printoptions(threshold=200)
np.set_printoptions(suppress=True)

warnings.filterwarnings('ignore')

In [8]:
from utilities.InterpretationNet import *
from utilities.LambdaNet import *
from utilities.metrics import *
from utilities.utility_functions import *
from utilities.DecisionTree_BASIC import *

#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

config['lambda_net']['number_of_lambda_weights'] = get_number_of_lambda_net_parameters(lambda_network_layers, number_of_variables, num_classes)
config['function_family']['basic_function_representation_length'] = (2 ** maximum_depth - 1) * number_of_variables + (2 ** maximum_depth - 1) + (2 ** maximum_depth) * num_classes
config['function_family']['function_representation_length'] = ( (2 ** maximum_depth - 1) * number_of_variables + (2 ** maximum_depth - 1) + (2 ** maximum_depth) * num_classes  if function_representation_type == 1 
                                                              else (2 ** maximum_depth - 1) * decision_sparsity + (2 ** maximum_depth - 1) + ((2 ** maximum_depth - 1)  * decision_sparsity * number_of_variables) + (2 ** maximum_depth) * num_classes if function_representation_type == 2
                                                              else (2 ** maximum_depth - 1) * decision_sparsity + ((2 ** maximum_depth - 1)  * decision_sparsity * number_of_variables) + (2 ** maximum_depth))

#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['i_net'])
globals().update(config['evaluation'])
globals().update(config['computation'])

#initialize_LambdaNet_config_from_curent_notebook(config)
#initialize_metrics_config_from_curent_notebook(config)
#initialize_utility_functions_config_from_curent_notebook(config)
#initialize_InterpretationNet_config_from_curent_notebook(config)


#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(config, path_type='interpretation_net'))
create_folders_inet(config)

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [9]:
print(path_identifier_interpretation_net)

print(path_identifier_lambda_net_data)


lNetSize1000_numLNets10000_var15_class2_random_vanilla_decision_tree_trained_xMax1_xMin0_xDistuniform_depth4_beta1_decisionSpars1_fullyGrown/256_e1000ES0.01_b64_drop0_adam_binary_crossentropy_fixedInit1-seed42/inet_dense2048_drop0e200b256_adam
lNetSize1000_numLNets10000_var15_class2_random_vanilla_decision_tree_trained_xMax1_xMin0_xDistuniform_depth4_beta1_decisionSpars1_fullyGrown/256_e1000ES0.01_b64_drop0_adam_binary_crossentropy_fixedInit1-seed42


In [10]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))

Num GPUs Available:  0
Num XLA-GPUs Available:  0


2021-09-07 08:31:35.651286: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-07 08:31:35.651324: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


## Load Data and Generate Datasets

In [11]:
def load_lambda_nets(config, no_noise=False, n_jobs=1):
    
    #def generate_lambda_net()
    
    #if psutil.virtual_memory().percent > 80:
        #raise SystemExit("Out of RAM!")
    
    if no_noise==True:
        config['noise_injected_level'] = 0
    path_dict = generate_paths(config, path_type='interpretation_net')        
        
    directory = './data/weights/' + 'weights_' + path_dict['path_identifier_lambda_net_data'] + '/'
    path_network_parameters = directory + 'weights' + '.txt'
    path_X_data = directory + 'X_test_lambda.txt'
    path_y_data = directory + 'y_test_lambda.txt'        
    
    network_parameters = pd.read_csv(path_network_parameters, sep=",", header=None)
    network_parameters = network_parameters.sort_values(by=0)
    if no_noise == False:
        network_parameters = network_parameters.sample(n=config['i_net']['interpretation_dataset_size'], random_state=config['computation']['RANDOM_SEED'])
    
    X_test_lambda = pd.read_csv(path_X_data, sep=",", header=None)
    X_test_lambda = X_test_lambda.sort_values(by=0)
    if no_noise == False:
        X_test_lambda = X_test_lambda.sample(n=config['i_net']['interpretation_dataset_size'], random_state=config['computation']['RANDOM_SEED'])
    
    y_test_lambda = pd.read_csv(path_y_data, sep=",", header=None)
    y_test_lambda = y_test_lambda.sort_values(by=0)
    if no_noise == False:
        y_test_lambda = y_test_lambda.sample(n=config['i_net']['interpretation_dataset_size'], random_state=config['computation']['RANDOM_SEED'])
        
        
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky') #loky

    lambda_nets = parallel(delayed(LambdaNet)(network_parameters_row, 
                                              X_test_lambda_row, 
                                              y_test_lambda_row, 
                                              config) for network_parameters_row, X_test_lambda_row, y_test_lambda_row in zip(network_parameters.values, X_test_lambda.values, y_test_lambda.values))          
    del parallel
    
    base_model = generate_base_model(config)  
    
    def initialize_network_wrapper(config, lambda_net, base_model):
        lambda_net.initialize_network(config, base_model)
    
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='sequential')
    _ = parallel(delayed(initialize_network_wrapper)(config, lambda_net, base_model) for lambda_net in lambda_nets)   
    del parallel
    
    def initialize_target_function_wrapper(config, lambda_net):
        lambda_net.initialize_target_function(config)
    
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='sequential')
    _ = parallel(delayed(initialize_target_function_wrapper)(config, lambda_net) for lambda_net in lambda_nets)   
    del parallel
        
    
    #lambda_nets = [None] * network_parameters.shape[0]
    #for i, (network_parameters_row, X_test_lambda_row, y_test_lambda_row) in tqdm(enumerate(zip(network_parameters.values, X_test_lambda.values, y_test_lambda.values)), total=network_parameters.values.shape[0]):        
    #    lambda_net = LambdaNet(network_parameters_row, X_test_lambda_row, y_test_lambda_row, config)
    #    lambda_nets[i] = lambda_net
                
    lambda_net_dataset = LambdaNetDataset(lambda_nets)
        
    return lambda_net_dataset
    

In [12]:
#LOAD DATA
if noise_injected_level > 0:
    lambda_net_dataset_training = load_lambda_nets(config, no_noise=True, n_jobs=n_jobs)
    lambda_net_dataset_evaluation = load_lambda_nets(config, n_jobs=n_jobs)

    lambda_net_dataset_train, lambda_net_dataset_valid = split_LambdaNetDataset(lambda_net_dataset_training, test_split=0.1)
    _, lambda_net_dataset_test = split_LambdaNetDataset(lambda_net_dataset_evaluation, test_split=test_size)
    
else:
    lambda_net_dataset = load_lambda_nets(config, n_jobs=n_jobs)

    lambda_net_dataset_train_with_valid, lambda_net_dataset_test = split_LambdaNetDataset(lambda_net_dataset, test_split=test_size)
    lambda_net_dataset_train, lambda_net_dataset_valid = split_LambdaNetDataset(lambda_net_dataset_train_with_valid, test_split=0.1)

    

2021-09-07 08:31:35.651349: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dws-05): /proc/driver/nvidia/version does not exist
[Parallel(n_jobs=-3)]: Using backend LokyBackend with 14 concurrent workers.
2021-09-07 08:32:12.770208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-07 08:32:12.770235: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-09-07 08:32:12.800779: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2021-09-07 08:32

## Data Inspection

In [13]:
lambda_net_dataset_train.shape

(8955, 4627)

In [14]:
lambda_net_dataset_valid.shape

(995, 4627)

In [15]:
lambda_net_dataset_test.shape

(50, 4627)

In [16]:
lambda_net_dataset_train.as_pandas(config).head()

,index,seed,f0v0,f0v1,f0v2,f0v3,f0v4,f0v5,f0v6,f0v7,f0v8,f0v9,f0v10,f0v11,f0v12,f0v13,f0v14,f1v0,f1v1,f1v2,f1v3,f1v4,f1v5,f1v6,f1v7,f1v8,f1v9,f1v10,f1v11,f1v12,f1v13,f1v14,f2v0,f2v1,f2v2,f2v3,f2v4,f2v5,f2v6,f2v7,f2v8,f2v9,f2v10,f2v11,f2v12,f2v13,f2v14,f3v0,f3v1,f3v2,f3v3,f3v4,f3v5,f3v6,f3v7,f3v8,f3v9,f3v10,f3v11,f3v12,f3v13,f3v14,f4v0,f4v1,f4v2,f4v3,f4v4,f4v5,f4v6,f4v7,f4v8,f4v9,f4v10,f4v11,f4v12,f4v13,f4v14,f5v0,f5v1,f5v2,f5v3,f5v4,f5v5,f5v6,f5v7,f5v8,f5v9,f5v10,f5v11,f5v12,f5v13,f5v14,f6v0,f6v1,f6v2,f6v3,f6v4,f6v5,f6v6,f6v7,...,wb_4253,wb_4254,wb_4255,wb_4256,wb_4257,wb_4258,wb_4259,wb_4260,wb_4261,wb_4262,wb_4263,wb_4264,wb_4265,wb_4266,wb_4267,wb_4268,wb_4269,wb_4270,wb_4271,wb_4272,wb_4273,wb_4274,wb_4275,wb_4276,wb_4277,wb_4278,wb_4279,wb_4280,wb_4281,wb_4282,wb_4283,wb_4284,wb_4285,wb_4286,wb_4287,wb_4288,wb_4289,wb_4290,wb_4291,wb_4292,wb_4293,wb_4294,wb_4295,wb_4296,wb_4297,wb_4298,wb_4299,wb_4300,wb_4301,wb_4302,wb_4303,wb_4304,wb_4305,wb_4306,wb_4307,wb_4308,wb_4309,wb_4310,wb_4311,wb_4312,wb_4313,wb_4314,wb_4315,wb_4316,wb_4317,wb_4318,wb_4319,wb_4320,wb_4321,wb_4322,wb_4323,wb_4324,wb_4325,wb_4326,wb_4327,wb_4328,wb_4329,wb_4330,wb_4331,wb_4332,wb_4333,wb_4334,wb_4335,wb_4336,wb_4337,wb_4338,wb_4339,wb_4340,wb_4341,wb_4342,wb_4343,wb_4344,wb_4345,wb_4346,wb_4347,wb_4348,wb_4349,wb_4350,wb_4351,wb_4352
6671,6671.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-0.118,0.228,-0.047,-0.259,-0.247,-1.019,0.363,0.245,0.202,-0.086,-0.276,0.271,-0.942,-0.234,-0.034,0.702,-0.067,-0.796,-0.295,-0.800,-0.232,1.238,-0.116,-0.925,-0.900,-0.175,0.607,0.267,-0.152,0.448,-0.042,-0.151,-0.142,-0.199,-0.205,1.049,-0.720,-0.307,0.729,0.233,0.231,-0.045,0.827,0.152,-0.166,-0.204,0.724,0.295,-0.080,1.156,0.213,0.026,-0.204,0.508,-0.218,0.439,1.287,0.706,0.268,0.249,-0.277,-0.188,-0.103,0.149,0.994,0.191,0.325,1.212,0.199,0.717,1.069,-0.024,0.043,0.242,-0.150,0.222,-0.221,0.381,0.286,-0.147,-0.055,-0.849,0.100,0.686,0.050,0.610,-0.067,-0.218,-0.159,-0.257,-0.212,0.284,-0.153,0.313,-0.673,0.064,-0.173,-0.248,0.957,0.014
3274,3274.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-0.293,0.237,-0.308,-0.224,-0.323,-1.536,0.913,0.702,0.204,-0.073,-0.315,0.237,-0.056,-0.281,-0.286,0.618,-0.351,-0.701,-0.688,-1.320,-0.220,0.129,-0.154,-1.502,-1.502,-0.264,0.219,0.413,-0.927,0.876,-0.042,-0.164,-0.172,0.245,0.259,0.060,-0.959,-0.030,0.485,0.477,0.313,-0.177,-0.262,0.206,-0.189,-0.254,0.416,0.314,-0.264,0.787,-0.267,0.240,-0.212,0.724,-0.553,0.292,0.649,0.307,0.345,0.392,-1.040,0.141,-0.282,0.566,0.462,-0.178,0.416,0.672,0.197,0.492,0.829,-0.273,-0.291,0.282,-0.150,0.399,0.303,0.354,0.354,-0.147,-0.652,-1.222,0.233,0.321,0.599,0.363,-0.393,-1.098,-0.164,-0.305,-0.254,0.293,-0.208,0.262,-0.399,0.181,-0.310,-0.390,0.473,0.147
3095,3095.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0

In [17]:
lambda_net_dataset_valid.as_pandas(config).head()

,index,seed,f0v0,f0v1,f0v2,f0v3,f0v4,f0v5,f0v6,f0v7,f0v8,f0v9,f0v10,f0v11,f0v12,f0v13,f0v14,f1v0,f1v1,f1v2,f1v3,f1v4,f1v5,f1v6,f1v7,f1v8,f1v9,f1v10,f1v11,f1v12,f1v13,f1v14,f2v0,f2v1,f2v2,f2v3,f2v4,f2v5,f2v6,f2v7,f2v8,f2v9,f2v10,f2v11,f2v12,f2v13,f2v14,f3v0,f3v1,f3v2,f3v3,f3v4,f3v5,f3v6,f3v7,f3v8,f3v9,f3v10,f3v11,f3v12,f3v13,f3v14,f4v0,f4v1,f4v2,f4v3,f4v4,f4v5,f4v6,f4v7,f4v8,f4v9,f4v10,f4v11,f4v12,f4v13,f4v14,f5v0,f5v1,f5v2,f5v3,f5v4,f5v5,f5v6,f5v7,f5v8,f5v9,f5v10,f5v11,f5v12,f5v13,f5v14,f6v0,f6v1,f6v2,f6v3,f6v4,f6v5,f6v6,f6v7,...,wb_4253,wb_4254,wb_4255,wb_4256,wb_4257,wb_4258,wb_4259,wb_4260,wb_4261,wb_4262,wb_4263,wb_4264,wb_4265,wb_4266,wb_4267,wb_4268,wb_4269,wb_4270,wb_4271,wb_4272,wb_4273,wb_4274,wb_4275,wb_4276,wb_4277,wb_4278,wb_4279,wb_4280,wb_4281,wb_4282,wb_4283,wb_4284,wb_4285,wb_4286,wb_4287,wb_4288,wb_4289,wb_4290,wb_4291,wb_4292,wb_4293,wb_4294,wb_4295,wb_4296,wb_4297,wb_4298,wb_4299,wb_4300,wb_4301,wb_4302,wb_4303,wb_4304,wb_4305,wb_4306,wb_4307,wb_4308,wb_4309,wb_4310,wb_4311,wb_4312,wb_4313,wb_4314,wb_4315,wb_4316,wb_4317,wb_4318,wb_4319,wb_4320,wb_4321,wb_4322,wb_4323,wb_4324,wb_4325,wb_4326,wb_4327,wb_4328,wb_4329,wb_4330,wb_4331,wb_4332,wb_4333,wb_4334,wb_4335,wb_4336,wb_4337,wb_4338,wb_4339,wb_4340,wb_4341,wb_4342,wb_4343,wb_4344,wb_4345,wb_4346,wb_4347,wb_4348,wb_4349,wb_4350,wb_4351,wb_4352
3466,3466.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-0.332,0.299,-0.435,-0.392,-0.377,-0.497,0.683,0.464,0.154,-0.088,-0.379,0.250,-0.050,-0.397,-0.347,0.048,-0.292,-0.029,-0.427,-0.507,-0.302,0.718,-0.403,-0.020,-0.470,-0.125,0.386,0.335,-0.152,0.512,-0.042,-0.282,-0.381,-0.324,-0.355,0.449,-0.432,-0.043,0.446,0.138,0.357,-0.246,-0.334,0.335,-0.266,-0.344,0.470,0.400,-0.408,0.106,-0.323,0.281,-0.279,0.144,-0.342,0.393,0.137,-0.396,0.420,0.332,-0.475,-0.322,-0.102,0.359,0.465,-0.273,0.357,0.397,0.325,0.387,0.060,-0.336,-0.385,0.391,-0.150,0.330,0.290,0.325,0.448,-0.147,-0.416,-0.361,0.241,-0.406,0.444,0.111,-0.061,-0.359,-0.327,-0.052,-0.385,0.358,-0.346,0.341,-0.437,0.205,-0.318,-0.113,0.402,0.015
689,689.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-0.223,0.076,-0.168,-0.168,-0.138,-0.133,0.062,0.074,0.124,-0.085,-0.075,0.106,-0.191,-0.086,-0.104,0.142,-0.206,-0.024,-0.093,-0.121,-0.117,0.264,-0.127,-0.026,-0.115,-0.083,0.142,0.123,-0.148,0.196,-0.037,-0.073,-0.163,-0.084,-0.081,0.175,-0.076,-0.025,0.194,0.115,-0.002,-0.085,-0.141,0.160,0.070,-0.122,0.259,0.093,-0.207,0.219,-0.096,0.079,0.067,0.238,-0.016,0.091,0.147,0.149,0.109,0.245,-0.102,-0.078,-0.244,0.221,0.114,-0.055,0.148,0.156,0.056,0.150,0.196,-0.151,-0.154,0.118,-0.150,0.178,-0.102,0.043,0.172,-0.142,-0.081,-0.002,0.062,-0.058,0.087,0.221,-0.174,-0.147,-0.182,-0.043,-0.125,0.174,0.118,0.073,-0.082,0.057,-0.153,-0.247,0.200,0.012
4148,4148.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0

In [18]:
lambda_net_dataset_test.as_pandas(config).head()

,index,seed,f0v0,f0v1,f0v2,f0v3,f0v4,f0v5,f0v6,f0v7,f0v8,f0v9,f0v10,f0v11,f0v12,f0v13,f0v14,f1v0,f1v1,f1v2,f1v3,f1v4,f1v5,f1v6,f1v7,f1v8,f1v9,f1v10,f1v11,f1v12,f1v13,f1v14,f2v0,f2v1,f2v2,f2v3,f2v4,f2v5,f2v6,f2v7,f2v8,f2v9,f2v10,f2v11,f2v12,f2v13,f2v14,f3v0,f3v1,f3v2,f3v3,f3v4,f3v5,f3v6,f3v7,f3v8,f3v9,f3v10,f3v11,f3v12,f3v13,f3v14,f4v0,f4v1,f4v2,f4v3,f4v4,f4v5,f4v6,f4v7,f4v8,f4v9,f4v10,f4v11,f4v12,f4v13,f4v14,f5v0,f5v1,f5v2,f5v3,f5v4,f5v5,f5v6,f5v7,f5v8,f5v9,f5v10,f5v11,f5v12,f5v13,f5v14,f6v0,f6v1,f6v2,f6v3,f6v4,f6v5,f6v6,f6v7,...,wb_4253,wb_4254,wb_4255,wb_4256,wb_4257,wb_4258,wb_4259,wb_4260,wb_4261,wb_4262,wb_4263,wb_4264,wb_4265,wb_4266,wb_4267,wb_4268,wb_4269,wb_4270,wb_4271,wb_4272,wb_4273,wb_4274,wb_4275,wb_4276,wb_4277,wb_4278,wb_4279,wb_4280,wb_4281,wb_4282,wb_4283,wb_4284,wb_4285,wb_4286,wb_4287,wb_4288,wb_4289,wb_4290,wb_4291,wb_4292,wb_4293,wb_4294,wb_4295,wb_4296,wb_4297,wb_4298,wb_4299,wb_4300,wb_4301,wb_4302,wb_4303,wb_4304,wb_4305,wb_4306,wb_4307,wb_4308,wb_4309,wb_4310,wb_4311,wb_4312,wb_4313,wb_4314,wb_4315,wb_4316,wb_4317,wb_4318,wb_4319,wb_4320,wb_4321,wb_4322,wb_4323,wb_4324,wb_4325,wb_4326,wb_4327,wb_4328,wb_4329,wb_4330,wb_4331,wb_4332,wb_4333,wb_4334,wb_4335,wb_4336,wb_4337,wb_4338,wb_4339,wb_4340,wb_4341,wb_4342,wb_4343,wb_4344,wb_4345,wb_4346,wb_4347,wb_4348,wb_4349,wb_4350,wb_4351,wb_4352
7217,7217.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-0.144,-0.006,-0.131,-0.107,-0.086,-0.130,0.090,0.033,0.121,-0.105,-0.123,0.097,-0.063,-0.112,-0.095,0.070,-0.065,-0.020,-0.116,-0.158,-0.020,0.135,-0.136,-0.082,-0.186,-0.076,0.088,0.097,-0.152,0.147,-0.042,-0.018,-0.096,0.007,-0.020,0.064,-0.081,-0.058,0.080,0.031,-0.012,-0.068,-0.041,0.095,-0.031,-0.053,0.170,0.111,-0.135,0.139,-0.010,0.063,-0.029,0.149,-0.050,0.110,0.145,0.005,0.082,0.138,-0.143,-0.042,-0.119,0.132,0.112,-0.018,0.012,0.005,0.047,0.022,0.106,-0.089,-0.039,0.118,-0.150,0.058,-0.050,0.048,0.042,-0.147,-0.099,0.019,0.066,0.002,0.080,0.128,-0.120,-0.041,-0.148,-0.106,-0.146,0.079,-0.005,0.085,-0.157,0.086,-0.109,-0.130,0.125,-0.016
8291,8291.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,-0.224,0.260,-0.058,-0.200,-0.331,-0.471,0.606,0.430,0.315,-0.441,-0.346,0.299,-0.054,-0.312,-0.320,0.015,-0.359,-0.022,-0.386,-0.433,-0.288,0.133,-0.433,-0.360,-0.492,-0.309,0.275,0.266,-0.152,0.331,-0.391,-0.172,-0.191,-0.268,-0.305,0.058,-0.381,-0.364,0.345,0.216,0.316,-0.050,-0.462,0.233,-0.198,-0.231,0.095,0.341,-0.080,0.101,0.235,0.239,-0.250,0.117,-0.293,0.339,0.458,-0.349,0.364,0.286,-0.381,-0.244,-0.282,0.355,0.387,-0.151,-0.377,0.003,0.253,-0.389,0.062,-0.182,-0.377,0.359,-0.150,0.222,-0.190,0.333,0.018,-0.147,-0.357,-0.348,0.313,0.299,0.044,0.101,-0.288,-0.246,-0.112,-0.398,-0.308,0.237,-0.302,0.347,-0.373,0.281,-0.041,-0.117,0.111,-0.053
4607,4607.000,42,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0

## Interpretation Network Training

In [19]:
#%load_ext autoreload

In [20]:
#%autoreload 2
((X_valid, y_valid), 
 (X_test, y_test),
 history,

 model) = interpretation_net_training(
                                      lambda_net_dataset_train, 
                                      lambda_net_dataset_valid, 
                                      lambda_net_dataset_test,
                                      config,
                                      #callback_names=['plot_losses']
                                     )



----------------------------------------------- TRAINING INTERPRETATION NET -----------------------------------------------


2021-09-07 08:36:06.501631: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/200
35/35 [==============================] - 28s 412ms/step - loss: 0.6610 - binary_accuracy_inet_decision_function_fv_metric: 0.6182 - val_loss: 0.6086 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6757
Epoch 2/200
35/35 [==============================] - 12s 351ms/step - loss: 0.6099 - binary_accuracy_inet_decision_function_fv_metric: 0.6716 - val_loss: 0.6034 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6797
Epoch 3/200
35/35 [==============================] - 12s 355ms/step - loss: 0.6131 - binary_accuracy_inet_decision_function_fv_metric: 0.6678 - val_loss: 0.6110 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6707
Epoch 4/200
35/35 [==============================] - 12s 355ms/step - loss: 0.6111 - binary_accuracy_inet_decision_function_fv_metric: 0.6686 - val_loss: 0.6117 - val_binary_accuracy_inet_decision_function_fv_metric: 0.6707
Epoch 5/200
35/35 [==============================] - 13s 364ms/step - loss: 0.6146 - binary_accuracy_ine

2021-09-07 08:57:34.533414: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Training Time: 0:21:35
---------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------ LOADING MODELS -----------------------------------------------------
Loading Time: 0:00:02


<Figure size 432x288 with 0 Axes>

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 4353)]       0                                            
__________________________________________________________________________________________________
hidden1_2048 (Dense)            (None, 2048)         8916992     input[0][0]                      
__________________________________________________________________________________________________
activation1_relu (Activation)   (None, 2048)         0           hidden1_2048[0][0]               
__________________________________________________________________________________________________
output_coeff_15 (Dense)         (None, 15)           30735       activation1_relu[0][0]           
______________________________________________________________________________________________

In [26]:
y_test_inet_vanilla_dt_list = []
y_test_distilled_sklearn_vanilla_dt_list = []

binary_crossentropy_distilled_sklearn_vanilla_dt_list =[]
accuracy_distilled_sklearn_vanilla_dt_list = []
f1_score_distilled_sklearn_vanilla_dt_list = []

binary_crossentropy_inet_vanilla_dt_list =[]
accuracy_inet_vanilla_dt_list = []
f1_score_inet_vanilla_dt_list = []

#inet_metric_function_list = []

number = lambda_net_dataset_test.y_test_lambda_array.shape[0]#10

for lambda_net_parameters, lambda_net, X_test_lambda, y_test_lambda in tqdm(zip(lambda_net_dataset_test.network_parameters_array[:number], lambda_net_dataset_test.network_list[:number], lambda_net_dataset_test.X_test_lambda_array[:number], lambda_net_dataset_test.y_test_lambda_array[:number]), total=lambda_net_dataset_test.y_test_lambda_array[:number].shape[0]):
    dt_inet = model.predict(np.array([lambda_net_parameters]))[0]
    
    X_data_random = generate_random_data_points_custom(0, 1, 500, 15)
    y_data_random_lambda_pred = lambda_net.predict(X_data_random)
    y_data_random_lambda_pred = np.round(y_data_random_lambda_pred).astype(np.int64)
    
    dt_sklearn_distilled = DecisionTreeClassifier(max_depth=config['function_family']['maximum_depth'])
    dt_sklearn_distilled.fit(X_data_random, y_data_random_lambda_pred)
    
    
    
    y_test_inet_vanilla_dt  = calculate_function_value_from_vanilla_decision_tree_parameters_wrapper(X_test_lambda, config)(dt_inet).numpy()
    y_test_distilled_sklearn_vanilla_dt = dt_sklearn_distilled.predict(X_test_lambda)
    
    y_test_lambda_pred = lambda_net.predict(X_test_lambda)
    y_test_lambda_pred = np.round(y_test_lambda_pred)
    
    
    #random_model = generate_base_model(config)        
    #random_network_parameters = random_model.get_weights()
    #network_parameters_structure = [network_parameter.shape for network_parameter in random_network_parameters]         
    
    #function_true_placeholder = np.array([0 for i in range(basic_function_representation_length)])
    #function_true_with_network_parameters = np.concatenate([function_true_placeholder, lambda_net_parameters])
    #inet_metric_function = inet_decision_function_fv_metric_wrapper(X_test_lambda, 
    #                                                                 random_model, 
    #                                                                 network_parameters_structure, 
    #                                                                 config, 
    #                                                                 'binary_accuracy')(np.array([function_true_with_network_parameters]), 
    #                                                                                     np.array([dt_inet]))    
    #inet_metric_function_list.append(inet_metric_function)
    
    
    binary_crossentropy_distilled_sklearn_vanilla_dt = log_loss(y_test_lambda_pred, y_test_distilled_sklearn_vanilla_dt, labels=[0,1])
    accuracy_distilled_sklearn_vanilla_dt = accuracy_score(y_test_lambda_pred, np.round(y_test_distilled_sklearn_vanilla_dt))
    f1_score_distilled_sklearn_vanilla_dt = f1_score(y_test_lambda_pred, np.round(y_test_distilled_sklearn_vanilla_dt))
    
    binary_crossentropy_inet_vanilla_dt = log_loss(y_test_lambda_pred, y_test_inet_vanilla_dt, labels=[0,1])
    accuracy_inet_vanilla_dt = accuracy_score(y_test_lambda_pred, np.round(y_test_inet_vanilla_dt))
    f1_score_inet_vanilla_dt = f1_score(y_test_lambda_pred, np.round(y_test_inet_vanilla_dt))
    
    
    y_test_inet_vanilla_dt_list.append(y_test_inet_vanilla_dt)
    y_test_distilled_sklearn_vanilla_dt_list.append(y_test_distilled_sklearn_vanilla_dt)    

    binary_crossentropy_distilled_sklearn_vanilla_dt_list.append(np.nan_to_num(binary_crossentropy_distilled_sklearn_vanilla_dt))
    accuracy_distilled_sklearn_vanilla_dt_list.append(np.nan_to_num(accuracy_distilled_sklearn_vanilla_dt))
    f1_score_distilled_sklearn_vanilla_dt_list.append(np.nan_to_num(f1_score_distilled_sklearn_vanilla_dt))

    binary_crossentropy_inet_vanilla_dt_list.append(np.nan_to_num(binary_crossentropy_inet_vanilla_dt))
    accuracy_inet_vanilla_dt_list.append(np.nan_to_num(accuracy_inet_vanilla_dt))
    f1_score_inet_vanilla_dt_list.append(np.nan_to_num(f1_score_inet_vanilla_dt))
    
y_test_inet_vanilla_dt_list = np.array(y_test_inet_vanilla_dt_list)
y_test_distilled_sklearn_vanilla_dt_list = np.array(y_test_distilled_sklearn_vanilla_dt_list)

binary_crossentropy_distilled_sklearn_vanilla_dt_list = np.array(binary_crossentropy_distilled_sklearn_vanilla_dt_list)
accuracy_distilled_sklearn_vanilla_dt_list = np.array(accuracy_distilled_sklearn_vanilla_dt_list)
f1_score_distilled_sklearn_vanilla_dt_list = np.array(f1_score_distilled_sklearn_vanilla_dt_list)

binary_crossentropy_inet_vanilla_dt_list = np.array(binary_crossentropy_inet_vanilla_dt_list)
accuracy_inet_vanilla_dt_list = np.array(accuracy_inet_vanilla_dt_list)
f1_score_inet_vanilla_dt_list = np.array(f1_score_inet_vanilla_dt_list)

inet_metric_function_list = np.array(inet_metric_function_list)
    

    
print('Binary Crossentropy:\t\t', np.round(np.mean(binary_crossentropy_distilled_sklearn_vanilla_dt_list), 3), '(Sklearn DT)' , '\t', np.round(np.mean(binary_crossentropy_inet_vanilla_dt_list), 3), '(I-Net DT)')
print('Accuracy:\t\t', np.round(np.mean(accuracy_distilled_sklearn_vanilla_dt_list), 3), '(Sklearn DT)' , '\t', np.round(np.mean(accuracy_inet_vanilla_dt_list), 3), '(I-Net DT)')
print('F1 Score:\t\t', np.round(np.mean(f1_score_distilled_sklearn_vanilla_dt_list), 3), '(Sklearn DT)' , '\t', np.round(np.mean(f1_score_inet_vanilla_dt_list), 3), '(I-Net DT)')
      

  0%|          | 0/50 [00:00<?, ?it/s]

Binary Crossentropy:		 4.526 (Sklearn DT) 	 0.554 (I-Net DT)
Accuracy:		 0.869 (Sklearn DT) 	 0.711 (I-Net DT)
F1 Score:		 0.83 (Sklearn DT) 	 0.608 (I-Net DT)
